# Problemas y Soluciones en el Proyecto RAG

Este notebook documenta cronológicamente los problemas enfrentados durante el desarrollo del proyecto de Retrieval-Augmented Generation (RAG) para consultas sobre discursos políticos, desde la configuración inicial hasta el despliegue final en Hugging Face. El objetivo es proporcionar una reflexión educativa sobre los desafíos comunes en proyectos de IA, con énfasis en el procesamiento de datos, integración de APIs y despliegue de aplicaciones web.

El proyecto involucra scraping de discursos de Casa Rosada, transcripción de audios, procesamiento de textos, configuración de un sistema RAG con ChromaDB y un modelo de lenguaje (Gemini), y una interfaz web con Streamlit.

**Estructura del notebook:**
- Configuración inicial e instalación de dependencias
- Scraping de datos
- Procesamiento de textos
- Configuración del sistema RAG
- Desarrollo de la interfaz web
- Despliegue en Hugging Face

Cada sección incluye problemas inferidos o comunes, soluciones aplicadas, ejemplos de código y referencias a interacciones previas basadas en los archivos del proyecto.

## 1. Configuración Inicial e Instalación de Dependencias

**Problema común:** Errores de instalación de paquetes debido a versiones incompatibles o entornos virtuales mal configurados. Por ejemplo, conflictos entre versiones de bibliotecas como `streamlit`, `chromadb` y `google-generativeai`.

**Solución:** Utilizar un entorno virtual con `venv` o `conda`, y especificar versiones exactas en `requirements.txt` para evitar conflictos. Ejecutar `pip install -r requirements.txt` después de activar el entorno.

**Ejemplo de código:**
```bash
python -m venv rag_env
rag_env\Scripts\activate  # En Windows
pip install -r requirements.txt
```

**Referencia:** Basado en el archivo `PROYECTO Base/requirements.txt`, que incluye dependencias como `streamlit==1.28.1`, `chromadb==0.4.18`, etc. Problemas iniciales pudieron surgir al no activar el entorno, causando errores de importación en scripts como `setup_gemini.py`.

## 2. Scraping de Datos

**Problema común:** Errores de red durante el scraping, como timeouts, bloqueos por rate limiting o cambios en la estructura HTML del sitio web (e.g., Casa Rosada). Además, problemas de parsing para extraer discursos correctamente.

**Solución:** Implementar manejo de excepciones con `try-except`, usar librerías como `requests` con headers para simular navegadores, y validar la estructura HTML con BeautifulSoup. Para lotes grandes, agregar delays entre requests.

**Ejemplo de código:**
```python
import requests
from bs4 import BeautifulSoup
import time

def scrape_discursos(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        discursos = soup.find_all('div', class_='discurso')
        time.sleep(1)  # Delay para evitar rate limiting
        return discursos
    except Exception as e:
        print(f'Error en scraping: {e}')
        return []
```

**Referencia:** Inspirado en `scraper_discursos.py` y notebooks como `Discursos_scrap2txt.ipynb`. Problemas pudieron incluir fallos en la conexión a `casarosada.gob.ar`, resolviendo con reintentos y validación de datos en `resultados_discursos/`.

## 3. Procesamiento de Textos

**Problema común:** Datos sucios o inconsistentes, como caracteres especiales, ruido en transcripciones de audio, o textos largos que exceden límites de tokens. Errores en la limpieza o segmentación de textos para embeddings.

**Solución:** Aplicar preprocesamiento con regex para limpiar textos, usar librerías como `nltk` o `spaCy` para tokenización, y dividir textos en chunks manejables. Validar la calidad de las transcripciones.

**Ejemplo de código:**
```python
import re
from nltk.tokenize import sent_tokenize

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remover espacios extra
    text = re.sub(r'[^\w\s]', '', text)  # Remover puntuación
    return text.lower()

def chunk_text(text, chunk_size=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ''
    for sentence in sentences:
        if len(current_chunk + sentence) > chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += ' ' + sentence
    if current_chunk:
        chunks.append(current_chunk)
    return chunks
```

**Referencia:** Basado en `generate_corpus.py` y `add_transcripts.py`, donde se procesan transcripciones de audio en `txt de transcripciones de audios para rag/`. Problemas comunes incluyen ruido en audios, resueltos con ajustes en la transcripción y limpieza en `mi_corpus.json`.

## 4. Configuración del Sistema RAG

**Problema común:** Errores en la configuración de la base de datos vectorial (ChromaDB), integración con APIs de modelos de lenguaje (e.g., Gemini), o problemas de caching y recuperación de embeddings. Fallos en la similitud semántica o timeouts en queries.

**Solución:** Configurar ChromaDB persistentemente, manejar claves API de forma segura, y optimizar la búsqueda con métricas de similitud. Implementar logging para debuggear queries.

**Ejemplo de código:**
```python
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path='./chroma_db')
collection = client.get_or_create_collection(name='discursos')

# Agregar documentos
collection.add(documents=chunks, ids=[f'id_{i}' for i in range(len(chunks))])

# Query
results = collection.query(query_texts=['consulta'], n_results=5)
```

**Referencia:** De `mvp_rag.py` y `create_vectorstore.py`, con ChromaDB en `PROYECTO Base/chroma_db/`. Problemas pudieron ser errores de API en `setup_gemini.py`, resueltos configurando variables de entorno y manejando excepciones en la integración con Gemini.

## 5. Desarrollo de la Interfaz Web con Streamlit

**Problema común:** Errores en la UI, como actualizaciones no reactivas, integración fallida con el backend RAG, o problemas de rendimiento con queries grandes. También, manejo de sesiones y estado.

**Solución:** Usar componentes de Streamlit para inputs y displays, integrar el RAG en funciones separadas, y optimizar con caching (`@st.cache_data`). Probar la app localmente antes de deploy.

**Ejemplo de código:**
```python
import streamlit as st
from mvp_rag import query_rag

st.title('Consultor RAG de Discursos')
query = st.text_input('Ingresa tu consulta:')
if st.button('Buscar'):
    response = query_rag(query)
    st.write(response)
```

**Referencia:** Basado en `app.py`, que es la aplicación Streamlit. Problemas comunes incluyen timeouts en queries, resueltos con indicadores de carga (`st.spinner`) y validación de inputs.

## 6. Despliegue en Hugging Face

**Problema común:** Errores en la configuración de Hugging Face Spaces, como incompatibilidades de dependencias, límites de recursos, o fallos en el runtime (e.g., GPU no disponible). También, problemas de privacidad con claves API.

**Solución:** Crear un Space con Streamlit, subir archivos necesarios (app.py, requirements.txt, chroma_db), y configurar secrets para APIs. Usar Docker si es necesario para entornos personalizados.

**Ejemplo de comandos:**
```bash
# En Hugging Face CLI o web
# Subir archivos al repo del Space
git add .
git commit -m 'Deploy RAG app'
git push origin main
```

**Referencia:** No hay archivos directos de deploy, pero inferido del contexto del proyecto. Problemas pudieron surgir con el tamaño de ChromaDB, resueltos optimizando el vectorstore o usando almacenamiento externo.

## Conclusión

Este proyecto RAG ilustra la importancia de una planificación cuidadosa y el manejo proactivo de errores en el desarrollo de IA. Desde la configuración inicial hasta el despliegue, cada etapa presenta desafíos únicos que requieren soluciones técnicas y reflexivas. Recomendamos documentar problemas en tiempo real para facilitar futuras iteraciones.

**Lecciones aprendidas:**
- Siempre validar datos en cada paso.
- Usar entornos controlados para dependencias.
- Implementar logging y excepciones robustas.
- Probar despliegues en entornos similares al producción.

Para más detalles, revisar los archivos del proyecto en `PROYECTO Base/`.